In [10]:
import pandas as pd
from IPython.display import display, HTML

In [11]:
dict_cities = {}
cities_file = open("corona_cities_apr_9.txt","r",encoding="utf-8-sig") # https://stackoverflow.com/questions/17912307/u-ufeff-in-python-string problem was found
index_ = 0
for line_ in cities_file:
    parts = line_.split()
    temp_ints = []
    temp_name = []
    for part_ in parts:
        if part_.isnumeric():
            temp_ints.append(part_)
        else:
            temp_name.append(part_)
    try:
        dict_cities[index_] = {'population': int(temp_ints[0]), 'sick': int(temp_ints[1]), 'city names': ' '.join(temp_name)}
        index_ = index_ + 1
    except IndexError:
        print(line_)
        break
print(dict_cities)

{0: {'population': 914559, 'sick': 1630, 'city names': 'ירושלים'}, 1: {'population': 195298, 'sick': 1594, 'city names': 'בני ברק'}, 2: {'population': 450192, 'sick': 415, 'city names': 'תל אביב - יפו'}, 3: {'population': 139032, 'sick': 216, 'city names': 'אשקלון'}, 4: {'population': 46760, 'sick': 215, 'city names': 'אלעד'}, 5: {'population': 242478, 'sick': 190, 'city names': 'פתח תקווה'}, 6: {'population': 241010, 'sick': 180, 'city names': 'ראשון לציון'}, 7: {'population': 73808, 'sick': 174, 'city names': 'מודיעין עילית'}, 8: {'population': 120812, 'sick': 169, 'city names': 'בית שמש'}, 9: {'population': 225073, 'sick': 160, 'city names': 'אשדוד'}, 10: {'population': 213971, 'sick': 149, 'city names': 'נתניה'}, 11: {'population': 185741, 'sick': 128, 'city names': 'חולון'}, 12: {'population': 196755, 'sick': 127, 'city names': 'באר שבע'}, 13: {'population': 143370, 'sick': 126, 'city names': 'רמת גן'}, 14: {'population': 44353, 'sick': 119, 'city names': 'טבריה'}, 15: {'populatio

In [12]:
cities_pretty = pd.DataFrame.from_dict(dict_cities, orient='index')
cities_pretty['wiki article'] = ["""https://nominatim.openstreetmap.org/search.php?q="""+str(i)+"""&format=json""" for i in cities_pretty['city names']]
cities_pretty['wiki'] = cities_pretty['wiki article'].apply(lambda x: '<a href="{0}">link</a>'.format(x))
display(HTML(cities_pretty.sort_values(by=['population'],ascending=False).head().to_html(escape=False)))

,population,sick,city names,wiki article,wiki
0,914559,1630,ירושלים,https://nominatim.openstreetmap.org/search.php?q=ירושלים&format=json,link
2,450192,415,תל אביב - יפו,https://nominatim.openstreetmap.org/search.php?q=תל אביב - יפו&format=json,link
15,272584,118,חיפה,https://nominatim.openstreetmap.org/search.php?q=חיפה&format=json,link
5,242478,190,פתח תקווה,https://nominatim.openstreetmap.org/search.php?q=פתח תקווה&format=json,link
6,241010,180,ראשון לציון,https://nominatim.openstreetmap.org/search.php?q=ראשון לציון&format=json,link


In [13]:
import requests
import json
import time

def get_geo(link):
    time.sleep(0.5)
    try:
        r = requests.get(link, timeout=1)
        if r.status_code  == 200:
            try:
                data = json.loads(r.content)
                if len(data[0]) > 1:
                    #display(data[0])
                    lat = data[0]['lat']
                    lon = data[0]['lon']
                    return lat, lon, True
                else:
                    return 0, 0, False
            except IndexError:
                return 0, 0, False
            except TypeError:
                return 0, 0, False
        else:
            return 0, 0, False
    except OSError:
        print("Failed to load {}".format(link))
        return 0, 0, False


In [14]:
print(cities_pretty['wiki article'][0])
get_geo(cities_pretty['wiki article'][0])

https://nominatim.openstreetmap.org/search.php?q=ירושלים&format=json


('31.778345', '35.2250786', True)

In [15]:
lats = []
lons = []
valids = []

import ipywidgets as widgets

bar = widgets.IntProgress(
    value=0,
    min=0,
    max=len(cities_pretty['wiki article']),
    step=1,
    description='Loading:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal'
)


display(bar)

for n, i in enumerate(cities_pretty['wiki article']):
    lat, lon, valid_geo = get_geo(i)
    lats.append(lat)
    lons.append(lon)
    valids.append(valid_geo)
    bar.value = (n+1)




IntProgress(value=0, description='Loading:', max=160)

In [16]:
cities_pretty['lat'] = lats
cities_pretty['lon'] = lons
cities_pretty['valid GEO'] = valids

In [17]:
cities_pretty['per 1000'] = cities_pretty['sick']/cities_pretty['population']*1000
display(HTML(cities_pretty.sort_values(by=['per 1000'],ascending=False).head(20).to_html(escape=False)))
cities_pretty['text'] = 'At:\n' + cities_pretty['city names'] +'\nSick: '+ cities_pretty['sick'].astype(str) + '\nPopulation: ' + cities_pretty['population'].astype(str)+ '\nPer 1000: ' + cities_pretty['per 1000'].astype(str)  
cities_pretty.sort_values(by=['population'],ascending=False).to_csv('CompleteCities.csv')
cities_pretty.sort_values(by=['population'],ascending=False).to_json('CompleteCities.json')

,population,sick,city names,wiki article,wiki,lat,lon,valid GEO,per 1000
1,195298,1594,בני ברק,https://nominatim.openstreetmap.org/search.php?q=בני ברק&format=json,link,32.0873899,34.8324376,True,8.161886
35,5713,38,קרית יערים,https://nominatim.openstreetmap.org/search.php?q=קרית יערים&format=json,link,31.8035128,35.1012545,True,6.651497
32,6724,43,"כפר חב""ד","https://nominatim.openstreetmap.org/search.php?q=כפר חב""ד&format=json",link,31.9907676,34.849123,True,6.395003
24,10676,64,אפרת,https://nominatim.openstreetmap.org/search.php?q=אפרת&format=json,link,31.6536462,35.1508714,True,5.994755
33,8602,42,כוכב יעקב,https://nominatim.openstreetmap.org/search.php?q=כוכב יעקב&format=json,link,31.8814286,35.2452887,True,4.882585
4,46760,215,אלעד,https://nominatim.openstreetmap.org/search.php?q=אלעד&format=json,link,32.0495306,34.9507087,True,4.597947
20,26058,92,מגדל העמק,https://nominatim.openstreetmap.org/search.php?q=מגדל העמק&format=json,link,32.6765744,35.2413237,True,3.530586
19,34843,102,אור יהודה,https://nominatim.openstreetmap.org/search.php?q=אור יהודה&format=json,link,32.0309712,34.8523936,True,2.927417
14,44353,119,טבריה,https://nominatim.openstreetmap.org/search.php?q=טבריה&format=json,link,32.7938522,35.5328566,True,2.683020
47,11841,28,רכסים,https://nominatim.openstreetmap.org/search.php?q=רכסים&format=json,link,32.7509558,35.1013589,True,2.364665


In [18]:
import plotly.express as px
import plotly.io as pio
df = cities_pretty[cities_pretty['valid GEO']==True]
fig = px.scatter_geo(df,
                     locationmode = 'country names',
                    lon = df['lon'],
                    lat = df['lat'],
                     hover_name="text", # column added to hover information
                     size="sick", # size of markers
                     color = "per 1000",
                     projection = 'natural earth',
                    #projection_scale= 0.1,
                    center = {'lat': 31.778345, 'lon':35.2250786})
fig.update_geos(
    showcountries=True, countrycolor="RebeccaPurple")

fig.show()
pio.write_html(fig,'index.html',auto_open=True)